# koelectra 사용 (https://github.com/monologg/KoELECTRA)

Data load / feature selection

일단, '요약문_연구목표', '요약문_연구내용', '요약문_기대효과' 를 사용해보기

In [1]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

2021-08-03 11:43:29.058926: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
# feather 파일 형식을 사용하기 위해 설치.
!pip install pyarrow

In [3]:
train_data = pd.read_feather("./data/re_train_data.feather")
test_data = pd.read_feather("./data/re_test_data.feather")


In [4]:
len(train_data), len(test_data)

(174304, 43576)

remake dataset to fit on BERT / ELECTRA / etc...

In [5]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

trainset / validset split

## tokenizer / model settings

In [6]:
from transformers import ElectraModel, ElectraTokenizer, ElectraForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [7]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator")
#tokenizer = AutoTokenizer.from_pretrained("")


# tokenizer = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=False, add_special_tokens=True, max_length=512, pad_to_max_length=True, return_attention_mask=True, truncation=True)
# model = RobertaForSequenceClassification.from_pretrained("roberta-base")

# |[CLS] '요약문_연구목표' [SEP] |'요약문_연구내용' [SEP] | '요약문_기대효과' [SEP]|
# |-----       64       -----|-----       128       -----|-----       64       -----|
가 되도록.

In [10]:
def f1_tokenizer(sent, MAX_LEN):
    encoded_dict=tokenizer.encode_plus(text = sent, add_special_tokens=True, max_length=MAX_LEN, pad_to_max_length=True, return_attention_mask=True, truncation = True)
	
    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    return input_id, attention_mask, token_type_id

def f2_tokenizer(sent, MAX_LEN):
    encoded_dict=tokenizer.encode_plus(text = sent, add_special_tokens=True, max_length=MAX_LEN+1, pad_to_max_length=True, return_attention_mask=True, truncation = True)
	
    input_id=encoded_dict['input_ids'][1:]
    attention_mask=encoded_dict['attention_mask'][1:]
    token_type_id = encoded_dict['token_type_ids'][1:]
    
    return input_id, attention_mask, token_type_id

In [11]:
input_ids =[]
attention_masks =[]
token_type_ids =[]

for train_obj in tqdm(train_data['요약문_연구목표']):
    input_id1, attention_mask1, token_type_id1 = f1_tokenizer(train_obj, MAX_LEN=64)
    
    
    input_ids.append(input_id1)
    attention_masks.append(attention_mask1)
    token_type_ids.append(token_type_id1)
    
for i, train_con in tqdm(enumerate(train_data['요약문_연구내용'])):
    input_id2, attention_mask2, token_type_id2 = f2_tokenizer(train_con, MAX_LEN=128)
    
    input_ids[i].extend(input_id2)
    attention_masks[i].extend(attention_mask2)
    token_type_ids[i].extend(token_type_id2)

for i, train_exp in tqdm(enumerate(train_data['요약문_기대효과'])):
    input_id3, attention_mask3, token_type_id3 = f2_tokenizer(train_exp, MAX_LEN=64)
    
    input_ids[i].extend(input_id3)
    attention_masks[i].extend(attention_mask3)
    token_type_ids[i].extend(token_type_id3)

  0%|          | 0/174304 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [12]:
test_input_ids =[]
test_attention_masks =[]
test_token_type_ids =[]

for test_obj in tqdm(test_data['요약문_연구목표']):
    input_id1, attention_mask1, token_type_id1 = f1_tokenizer(test_obj, MAX_LEN=64)
    
    
    test_input_ids.append(input_id1)
    test_attention_masks.append(attention_mask1)
    test_token_type_ids.append(token_type_id1)

    
for i, test_con in tqdm(enumerate(test_data['요약문_연구내용'])):
    input_id2, attention_mask2, token_type_id2 = f2_tokenizer(test_con, MAX_LEN=128)
    
    test_input_ids[i].extend(input_id2)
    test_attention_masks[i].extend(attention_mask2)
    test_token_type_ids[i].extend(token_type_id2)

for i, test_exp in tqdm(enumerate(test_data['요약문_기대효과'])):
    input_id3, attention_mask3, token_type_id3 = f2_tokenizer(test_exp, MAX_LEN=64)
    
    test_input_ids[i].extend(input_id3)
    test_attention_masks[i].extend(attention_mask3)
    test_token_type_ids[i].extend(token_type_id3)

  0%|          | 0/43576 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [13]:
train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(input_ids, train_data['label'].values, random_state=42, test_size=0.1)
train_masks, valid_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.1)
train_token_ids, valid_token_ids, _, _ = train_test_split(token_type_ids, input_ids, random_state=42, test_size=0.1)

In [14]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_ids = torch.tensor(train_token_ids)
valid_inputs = torch.tensor(valid_inputs)
valid_labels = torch.tensor(valid_labels)
valid_masks = torch.tensor(valid_masks)
valid_token_ids = torch.tensor(valid_token_ids)

In [15]:
BATCH_SIZE = 16

trainset = TensorDataset(train_inputs, train_masks, train_token_ids, train_labels)
train_sampler = RandomSampler(trainset)
train_dataloader = DataLoader(trainset, sampler=train_sampler, batch_size=BATCH_SIZE)

validset = TensorDataset(valid_inputs, valid_masks, valid_token_ids, valid_labels)
valid_sampler = SequentialSampler(validset)
valid_dataloader = DataLoader(validset, sampler=valid_sampler, batch_size=BATCH_SIZE)

In [16]:
test_inputs = torch.tensor(test_input_ids)
test_masks = torch.tensor(test_attention_masks)
test_token_ids = torch.tensor(test_token_type_ids)

testset = TensorDataset(test_inputs, test_masks, test_token_ids)
test_sampler = SequentialSampler(testset)
test_dataloader = DataLoader(testset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [17]:
device = torch.device("cuda:6")

In [18]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v2-discriminator", num_labels=46)
model.to(device)

Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [19]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값
epochs = 10
# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs
# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [20]:
# 정확도 계산 함수
from sklearn.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    #return np.sum(pred_flat == labels_flat) / len(labels_flat)
    return fl_score(pred_flat, labels_flat, average='macro')
    
# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [21]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_ids, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=b_token_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in valid_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_ids, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=b_token_ids, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
        
    avg_valid_loss = logits / len(valid_dataloader)            

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Average validation loss: {0:.2f}".format(avg_train_loss))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    if not best_val_loss or avg_valid_loss < best_val_loss:
	    if not os.path.isdir("model_weights"):
            os.makedirs("model_weights")
        best_epoch = e
        torch.save(model.state_dict(), './model_weights/koelectra_model_{}.pt'.format(e))
        best_val_loss = val_epoch_loss



print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...
  Batch   500  of  9,805.    Elapsed: 0:08:32.
  Batch 1,000  of  9,805.    Elapsed: 0:23:58.
  Batch 1,500  of  9,805.    Elapsed: 0:39:24.
  Batch 2,000  of  9,805.    Elapsed: 0:54:50.
  Batch 2,500  of  9,805.    Elapsed: 1:10:16.
  Batch 3,000  of  9,805.    Elapsed: 1:25:43.
  Batch 3,500  of  9,805.    Elapsed: 1:41:09.
  Batch 4,000  of  9,805.    Elapsed: 1:56:35.
  Batch 4,500  of  9,805.    Elapsed: 2:12:00.
  Batch 5,000  of  9,805.    Elapsed: 2:27:26.
  Batch 5,500  of  9,805.    Elapsed: 2:42:52.
  Batch 6,000  of  9,805.    Elapsed: 2:58:17.
  Batch 6,500  of  9,805.    Elapsed: 3:13:43.
  Batch 7,000  of  9,805.    Elapsed: 3:29:09.
  Batch 7,500  of  9,805.    Elapsed: 3:44:34.
  Batch 8,000  of  9,805.    Elapsed: 4:00:00.
  Batch 8,500  of  9,805.    Elapsed: 4:15:25.
  Batch 9,000  of  9,805.    Elapsed: 4:30:51.
  Batch 9,500  of  9,805.    Elapsed: 4:46:16.

  Average training loss: 0.65
  Training epcoh took: 4:55:40



In [25]:
len(outputs[0])   # batch_size만큼, 마지막 batch라 16이 아니라, 남은 숫자만큼 나옴.

7

In [28]:
len(outputs[0][0]) # 확률 값

46

In [30]:
outputs[0][0]

tensor([ 10.3797,  -5.5306,  -7.1882, -10.0134, -15.0338,  -3.8773, -10.6498,
        -13.2892,  -8.1916, -11.3695,  -7.0370,  -8.0767,  -8.1087,  -5.4747,
         -4.1041, -11.1189,  -4.5186,  -7.5130,  -4.2910,  -0.9708,  -3.3784,
         -8.4583, -10.3509,  -5.5739,  -5.1307,  -6.1944,  -8.6184,  -8.1003,
         -6.4558,  -4.8816, -12.8358,  -7.0732,  -8.9400,  -6.6901,  -6.9372,
         -8.2235,  -5.1464,  -8.5271,  -9.4629, -16.7443,  -8.2850,  -8.1729,
         -5.5341,  -8.9708, -12.2611,  -2.7458], device='cuda:6')

In [59]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.load_state_dict(torch.load('./model_weights/koelectra_model_{}.pt'.format(best_epoch)))
model.eval()

# 변수 초기화
nb_eval_steps, nb_eval_examples = 0, 0
y_pred_list = []

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 300 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_token_ids = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, token_type_ids=b_token_ids, attention_mask=b_input_mask)
    
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    
    y_pred_list.append(np.argmax(logits, axis=1).flatten())


print("")
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  2,724.    Elapsed: 0:00:09.
  Batch   200  of  2,724.    Elapsed: 0:00:18.
  Batch   300  of  2,724.    Elapsed: 0:00:27.
  Batch   400  of  2,724.    Elapsed: 0:00:37.
  Batch   500  of  2,724.    Elapsed: 0:00:46.
  Batch   600  of  2,724.    Elapsed: 0:00:56.
  Batch   700  of  2,724.    Elapsed: 0:01:13.
  Batch   800  of  2,724.    Elapsed: 0:02:04.
  Batch   900  of  2,724.    Elapsed: 0:03:08.
  Batch 1,000  of  2,724.    Elapsed: 0:04:11.
  Batch 1,100  of  2,724.    Elapsed: 0:05:15.
  Batch 1,200  of  2,724.    Elapsed: 0:06:19.
  Batch 1,300  of  2,724.    Elapsed: 0:07:22.
  Batch 1,400  of  2,724.    Elapsed: 0:08:26.
  Batch 1,500  of  2,724.    Elapsed: 0:09:30.
  Batch 1,600  of  2,724.    Elapsed: 0:10:33.
  Batch 1,700  of  2,724.    Elapsed: 0:11:37.
  Batch 1,800  of  2,724.    Elapsed: 0:12:41.
  Batch 1,900  of  2,724.    Elapsed: 0:13:44.
  Batch 2,000  of  2,724.    Elapsed: 0:14:48.
  Batch 2,100  of  2,724.    Elapsed: 0:15:51.
  Batch 2,200

In [63]:
y_pred_list[:10]

[array([ 0,  0,  0, 19,  0,  0, 18,  0,  0, 14,  0,  0, 23,  0,  0,  0]),
 array([ 0,  0,  0,  0, 27,  0,  0,  0, 24,  0,  0, 20,  0,  0,  0,  0]),
 array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 29,  0,  0,  0,  0,  0]),
 array([ 0, 29,  0,  0,  1,  0,  5, 14,  0,  0,  0,  0,  0,  0, 43, 19]),
 array([ 0,  0,  0,  0, 23,  0,  0,  0,  0,  0,  0,  0,  0,  0, 19,  0]),
 array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 25, 18,  0,  0,  0,  0]),
 array([ 0,  0,  0,  0,  0,  0,  0,  0, 43,  0,  0,  0,  0,  0,  0, 31]),
 array([ 0,  0,  0,  0,  0,  0,  0,  0,  0, 45,  0,  0,  0,  0,  0,  0]),
 array([ 0,  0,  0,  0,  0,  0,  0, 14, 27,  0,  0,  0,  0,  0,  0,  0]),
 array([ 0,  0,  0,  0,  0,  0,  0,  0, 23,  0,  0,  0,  0,  0,  0,  0])]

In [46]:
len(np.concatenate(y_pred_list))

43576

In [64]:
pred_result = np.concatenate(y_pred_list)

In [52]:
len(test_data), len(pred_result)

(43576, 43576)

In [65]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['label']= pred_result

In [69]:
submission.label.value_counts()

0     36568
19      978
24      737
14      436
5       417
23      386
18      325
1       281
29      250
45      244
36      243
31      221
20      206
13      169
25      168
27      159
10      145
16      133
21      132
34      123
40      116
28      108
33      104
8        77
43       73
38       68
2        67
12       61
30       60
35       59
37       54
32       52
26       50
11       44
22       37
7        35
15       32
39       31
9        29
3        27
44       23
6        16
17       16
4        11
42        5
Name: label, dtype: int64

In [67]:
submission.to_csv("./data/koelectra_256_10epochs.csv", index=False)

In [68]:
pd.read_csv("./data/koelectra_256_10epochs.csv")

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,19
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,2
43574,217878,0
